In [ ]:
#import 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import csv

In [ ]:
#launch url
url = "https://jobs.citi.com/search-jobs"

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

#cookie_button = driver.find_element_by_id('gdpr-button')
#cookie_button.click()

In [ ]:
python_button = 0
url_list = []

for page in range(1,192):
    soup_level1=BeautifulSoup(driver.page_source, 'lxml')
    
    for link in soup_level1.find_all( 'a', href=re.compile("/job//*")):
        url_list.append(link.get('href'))
        
    next_button = driver.find_element_by_xpath("//a[@class ='next']")
    next_button.click()
    time.sleep(10)

print(url_list)
driver.quit()

In [ ]:
with open('citigroup_jobs.csv', mode='w', encoding="utf-8") as csv_file:
    fieldnames = ['job_url', 'title', 'job_id', 'job_function', 'schedule', 'location', 'description']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for job_url in url_list:
        company_url = "https://jobs.citi.com"
        visit_url = company_url + job_url
        #visit_url ="https://jobs.citi.com/job/tampa/supervisory-principal/287/10788479"
        driver = webdriver.Chrome()
        driver.get(visit_url)
        soup =BeautifulSoup(driver.page_source, 'lxml')

        title = soup.find('h2', itemprop = True)
        if str(title) == 'None':
            title = soup.find('h1', {'class':'ajd_header__job-title'})

        job_id = soup.find('span', {'class':'job-description2 job-info'})
        if str(job_id) == 'None':
            job_id = soup.find('div',{'class':'ats-description ajd_job-details__ats-description'})
        job_id_lvl1 = job_id.contents[1]
        job_id_lvl2 = job_id_lvl1.contents[7]
        job_id_lvl3 = job_id_lvl2.contents[1]

        location_lvl1 = job_id_lvl1.contents[0]
        location_lvl2 = location_lvl1.contents[1]

        job_function_lvl1 = job_id.contents[1]
        job_function_lvl2 = job_function_lvl1.contents[2]
        job_function_lvl3 = job_function_lvl2.contents[1]

        schedule_lvl1 = job_id.contents[1]
        schedule_lvl2 = job_function_lvl1.contents[3]
        schedule_lvl3 = schedule_lvl2.contents[1]

        desc = soup.find('div', {'class':'ats-description'})

        writer.writerow({'job_url':visit_url, 'title':title.get_text(), 'job_id':job_id_lvl3, 'job_function':job_function_lvl3, 'schedule':schedule_lvl3, 'location':location_lvl2, 'description':desc.get_text()})
        driver.quit()